In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from pyRealParser import Tune
import json

%matplotlib inline

In [3]:
# Source: https://www.irealb.com/forums/showthread.php?12753-Jazz-1300-Standards
file = open('all_jazz_standards_ireal_link.txt', 'r')
ALL_JAZZ_STANDARDS_IREAL_LINK = file.readlines()[0]

In [4]:
tunes = Tune.parse_ireal_url(ALL_JAZZ_STANDARDS_IREAL_LINK)

Parsed 26-2
Parsed 500 Miles High
Parsed 502 Blues
Parsed 52nd Street Theme
Parsed 9.20 Special
Parsed A Ballad
Parsed A Beautiful Friendship
Parsed A Blossom Fell
Parsed A Certain Smile
Parsed A Child Is Born
Parsed A Felicidade
Parsed A Fine Romance
Parsed A Flower Is A Lovesome Thing
Parsed A Foggy Day
Parsed A Ghost Of A Chance
Parsed A Kiss To Build A Dream On
Parsed A Little Tear
Parsed A Lovely Way To Spend An Evening
Parsed A Night In Tunisia
Parsed A Nightingale Sang In Berkeley Square
Parsed A Pretty Girl Is Like A Melody
Parsed A Sleepin' Bee
Parsed A Smooth One
Parsed A Sound For Sore Ears
Parsed A Sunday Kind Of Love
Parsed A Taste Of Honey
Parsed A Time For Love
Parsed A Tisket A Tasket
Parsed A Weaver Of Dreams
Parsed A Wonderful Day Like Today
Parsed Ablution
Parsed Ac-Cent-Tchu-Ate The Positive
Parsed Across The Alley From The Alamo
Parsed Adam's Apple
Parsed Affirmation
Parsed African Queen, The
Parsed Afro Blue
Parsed Afro Centric
Parsed After You
Parsed After You've

Parsed Moonlight Serenade
Parsed Moontrane, The
Parsed Moose The Mooche
Parsed More I See You, The
Parsed More Than You Know
Parsed Morgan The Pirate
Parsed Morning
Parsed Moten Swing
Parsed Mountain Greenery
Parsed Move
Parsed Mr. P.C.
Parsed My Baby Just Cares For Me
Parsed My Buddy
Parsed My Favorite Things
Parsed My Foolish Heart
Parsed My Funny Valentine
Parsed My Heart Belongs To Daddy
Parsed My Heart Stood Still
Parsed My Ideal
Parsed My Last Affair
Parsed My Little Brown Book
Parsed My Little Suede Shoes
Parsed My Lucky Star
Parsed My Man's Gone Now
Parsed My Melancholy Baby
Parsed My Old Flame
Parsed My One And Only Love
Parsed My Romance
Parsed My Secret Love
Parsed My Shining Hour
Parsed My Ship
Parsed My Song
Parsed My Way
Parsed Nacada
Parsed Naima
Parsed Nancy (With The Laughing Face)
Parsed Nardis
Parsed Nascimento
Parsed Nature Boy
Parsed Nearness Of You, The
Parsed Nefertiti
Parsed Never Let Me Go
Parsed Never Will I Marry
Parsed Nevertheless
Parsed New Picture
Parsed 

In [5]:
len(tunes)

1350

In [16]:
autumn_leaves = tunes[95]
autumn_leaves

Title: Autumn Leaves
Composer: Kosma Joseph
Style: Medium Swing
Key: G-
Transpose: None
Comp style: 0
BPM: 0
Repeats: None
Time signature: 4/4

Chord string:
*A{T44C-7 |F7 |Bb^7 |Eb^7 |Ah7 |D7b13 |G-6 |x }*B[Ah7 |D7b13 |G-6 |x |C-7 |F7 |Bb^7 |Eb^7 ]*C[Ah7 |D7b13 |G-7 Gb7|F-7 E7|Ah7 |D7b13 |G-6 |x

Flattened measures:
| C-7         | F7          | Bb^7        | Eb^7        |
| Ah7         | D7b13       | G-6         | G-6         |
| C-7         | F7          | Bb^7        | Eb^7        |
| Ah7         | D7b13       | G-6         | G-6         |
| Ah7         | D7b13       | G-6         | G-6         |
| C-7         | F7          | Bb^7        | Eb^7        |
| Ah7         | D7b13       | G-7Gb7      | F-7E7       |
| Ah7         | D7b13       | G-6         | G-6         |

In [37]:
rhythm_changes = [tune for tune in tunes if 'I Got Rhythm' in tune.title][0]

# GOAL 1: Convert every song into a list of *individual* chords, all in the Key of C (if major) or Am (if minor)

### Split chords squished into single measures

In [39]:
def get_formatted_chords(tune):
    chords = []
    for measure in tune.measures_as_strings:
        print(measure) 
    # remove duplicate chords (so D7-D7-G matches D7-G)
    
    return chords

get_formatted_chords(rhythm_changes)

Bb6G-7
C-7F7
D-7G-7
C-7F7
Bb7Bb7/D
Eb7Eo7
Bb6/FF7
Bb6F7
Bb6G-7
C-7F7
D-7G-7
C-7F7
Bb7Bb7/D
Eb7Eo7
Bb6/FF7
Bb6
D7
D7
G7
G7
C7
C7
F7
F7
Bb6G-7
C-7F7
D-7G-7
C-7F7
Bb7Bb7/D
Eb7Eo7
Bb6/FF7
Bb6F7Bb6G-7
C-7F7
D-7G-7
C-7F7
Bb7Bb7/D
Eb7Eo7
Bb6/FF7
Bb6F7
Bb6G-7
C-7F7
D-7G-7
C-7F7
Bb7Bb7/D
Eb7Eo7
Bb6/FF7
Bb6
D7
D7
G7
G7
C7
C7
F7
F7
Bb6G-7
C-7F7
D-7G-7
C-7F7
Bb7Bb7/D
Eb7Eo7
Bb6/FEb9
D-7G7
C-7F7
Bb6


[]

In [55]:
a = 'Bb7Bb7/D'
b = 'Bb6/FF7'
c = 'Eb7Eo7'
d = 'Bb7Bb7/Db'
test_strs = [a,b,c,d]
# Naive: split at uppercase letter
def split_measure_into_chords(measure):   
    import re
    chunks = re.split(r'(\/[A-G])', measure)
    for chunk in chunks:
        print(chunk)
        # HANDLE SLASH CHORDS
        # If it starts with '/', add any lowercase letter to the right, and, and add it to the chord that comes befre
        # Otherwise, split at UpperCase letters
        
split_measure_into_chords(c)

Eb7Eo7


In [10]:
chords_arr = autumn_leaves.measures_as_strings
chords_arr

['C-7',
 'F7',
 'Bb^7',
 'Eb^7',
 'Ah7',
 'D7b13',
 'G-6',
 'G-6',
 'C-7',
 'F7',
 'Bb^7',
 'Eb^7',
 'Ah7',
 'D7b13',
 'G-6',
 'G-6',
 'Ah7',
 'D7b13',
 'G-6',
 'G-6',
 'C-7',
 'F7',
 'Bb^7',
 'Eb^7',
 'Ah7',
 'D7b13',
 'G-7Gb7',
 'F-7E7',
 'Ah7',
 'D7b13',
 'G-6',
 'G-6']

In [21]:
pd.Series([tune.key for tune in tunes]).unique()

array(['F', 'E-', 'A-', 'C', 'Bb', 'Db', 'D-', 'Eb', 'G', 'Ab', 'B-',
       'C-', 'F-', 'Bb-', 'D', 'G-', 'G#-', 'Eb-', 'B', 'E', 'C#-', 'A',
       'Gb'], dtype=object)

## Export a song

In [13]:
with open('autumn_leaves.json', 'w') as outfile:
    json.dump({'chords_arr': chords_arr}, outfile)